# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# %load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
#import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

# pgconfig = {
#     'host': 'db',
#     'port': os.environ['PG_PORT'],
#     'database': os.environ['PG_DATABASE'],
#     'user': os.environ['PG_USER'],
#     'password': os.environ['PG_PASSWORD'],
# }
# dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# # pd.read_sql用のコネクタ
# conn = psycopg2.connect(**pgconfig)
# # pd.to_sql用のcreate engine
# engine = create_engine(dsl)
# # MagicコマンドでSQLを書くための設定
# %sql $dsl

# df_customer = pd.read_sql(sql='select * from customer', con=conn)
# df_category = pd.read_sql(sql='select * from category', con=conn)
# df_product = pd.read_sql(sql='select * from product', con=conn)
# df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
# df_store = pd.read_sql(sql='select * from store', con=conn)
# df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\301518\AppData\Local\Continuum\anaconda3\envs\Cafetalk_1\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [5]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [7]:
df_geocode.head()

,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,NaN,NaN,NaN,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,NaN,NaN,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,NaN,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,NaN,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,NaN,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086


# 演習問題

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [8]:
pd.concat([df_customer['customer_id'],
           pd.to_datetime(df_customer['application_date'].astype('str'))], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [9]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_ymd'].astype('str'))],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [10]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009-11-03
1,1132,2,2009-11-18
2,1102,1,2008-07-12
3,1132,1,2010-02-05
4,1102,2,2009-08-21
5,1112,1,2010-06-05
6,1102,2,2009-12-05
7,1102,1,2010-09-22
8,1112,2,2008-05-04
9,1102,1,2010-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [11]:
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009
1,1132,2,2009
2,1102,1,2008
3,1132,1,2010
4,1102,2,2009
5,1112,1,2010
6,1102,2,2009
7,1102,1,2010
8,1112,2,2008
9,1102,1,2010


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [12]:
# dt.monthでも月を取得できるが、ここでは0埋め２桁で取り出すためstrftimeを利用している
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m')],axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10
